In [ ]:
missing_indicators = {
    'Year': [], 
    'Loan Number': [], 
    'FHLBankID': [], 
    'Program': [], 
    'FIPSStateCode': [],
    'FIPSCountyCode': [], 
    # 'MSA': [99999], 
    'MSA': [],
    'FeatureID': [], 
    'Tract': [], 
    'MinPer': [], 
    'TraMedY': [],
    'LocMedY': [], 
    'Tractrat': [],
    'Income': [], 
    'CurAreY': [], 
    'IncRat': [], 
    'UPB': [], 
    'LTV': [],
    'MortDate': [],
    'AcquDate': [], 
    'Purpose': [], 
    'Coop': [], 
    'Product': [], 
    'FedGuar': [], 
    'Term': [],
    'AmorTerm': [998], 
    'FHFBID': [], 
    'Seller': [], 
    'SellCity': [], 
    'SellSt': [], 
    'SellType': [],
    'NumBor': [], 
    'First': [2], 
    'CICA': [], 
    'BoRace': [], 
    'CoRace': [], 
    'BoGender': [], 
    'CoGender': [],
    'BoAge': [999, 99], 
    'CoAge': [999, 99], 
    'Occup': [], 
    'NumUnits': [], 
    'Bed1': [],
    'Bed2': [], 
    'Bed3': [], 
    'Bed4': [],
    'Aff1': [], 
    'Aff2': [], 
    'Aff3': [], 
    'Aff4': [], 
    # 'Rent1': [9999999999], 
    # 'Rent2': [9999999999],
    # 'Rent3': [9999999999], 
    # 'Rent4': [9999999999],
    # 'RentUt1': [9999999999], 
    # 'RentUt2': [9999999999], 
    # 'RentUt3': [9999999999], 
    # 'RentUt4': [9999999999],
    'Rent1': [], 
    'Rent2': [],
    'Rent3': [], 
    'Rent4': [],
    'RentUt1': [], 
    'RentUt2': [], 
    'RentUt3': [], 
    'RentUt4': [], 
    'Geog': [], 
    'Rate': [], 
    'Amount': [],
    'Front': [], 
    'Back': [], 
    'Borrower Credit Score': [9], 
    'Co-Borrower Credit Score': [],
    'PMI': [], 
    'Self': [0], 
    'PropType': [], 
    'ArmIndex': [], 
    'ArmMarg': [99999], 
    'PrepayP': ['12/31/9999', '9999-12-31 00:00:00', '9999-12-31'], 
    'BoEth': [3],
    'Race2': [6], 
    'Race3': [6], 
    'Race4': [6], 
    'Race5': [6], 
    'CoEth': [3], 
    'Corace2': [6], 
    'Corace3': [6],
    'Corace4': [6], 
    'Corace5': [6], 
    'HOEPA': [], 
    'LienStatus': [], 
    'SpcHsgGoals': [],
    'FedFinStbltyPlan': [], 
    'AcqTyp': [], 
    'GSEREO': [],
}

feature_to_drop = [
  "Loan Number",
  "Program",
  "FeatureID",
  "Tractrat",
  "IncRat",
  "Coop",
  "FedGuar",
  "AmorTerm",
  "FHFBID",
  "Seller",
  "SellCity",
  "SellSt",
  "SellType",
  "First",
  "CICA",
  "Occup",
  "Product",
  "NumUnits",
  "Bed1",
  "Bed2",
  "Bed3",
  "Bed4",
  "Aff1",
  "Aff2",
  "Aff3",
  "Aff4",
  "Rent1",
  "Rent2",
  "Rent3",
  "Rent4",
  "RentUt1",
  "RentUt2",
  "RentUt3",
  "RentUt4",
  "Geog",
  "PMI",
  "PropType",
  "ArmIndex",
  "ArmMarg",
  "PrepayP",
  "BoEth",
  "Race2",
  "Race3",
  "Race4",
  "Race5",
  "CoEth",
  "Corace2",
  "Corace3",
  "Corace4",
  "Corace5",
  "HOEPA",
  "LienStatus",
  "SpcHsgGoals",
  "FedFinStbltyPlan",
  "AcqTyp",
  "GSEREO",
]

import pandas as pd
def replace_missing_with_nan(df, missing_indicators):
    df_copy = df.copy()
    
    for column, missing_values in missing_indicators.items():
        if column in df_copy.columns:
            df_copy[column] = df_copy[column].replace(missing_values, np.nan)
    
    return df_copy

df = pd.read_csv('Yearly/merged_data.csv')
df = df.drop(columns=[col for col in feature_to_drop if col in df.columns])
print("Columns removed successfully. Remaining columns:")
print(df.columns)
df = replace_missing_with_nan(df, missing_indicators)
df.to_csv("Data/merged_data_dropped.csv", index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('Data/merged_data_dropped.csv', low_memory=False)
df = df.dropna(subset=['BoAge', 'CoAge', 'Borrower Credit Score'])
print('Removed rows with missing values of subset=[BoAge, CoAge, Borrower Credit Score]')

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
features = [col for col in df.columns if col not in ['FHLBankID']]
known_data = df[df['FHLBankID'].notnull()]
X_train, X_val, y_train, y_val = train_test_split(known_data[features], known_data['FHLBankID'], test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
X_test = df[df['FHLBankID'].isnull()][features]
df.loc[df['FHLBankID'].isnull(), 'FHLBankID'] = rf_model.predict(X_test)
print(f"Imputed FHLBankID with RandomForestClassifier with Validation Accuracy : {accuracy:.4f}")

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.loc[df["FHLBankID"].notnull(), "FHLBankID"] = le.fit_transform(df.loc[df["FHLBankID"].notnull(), "FHLBankID"])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Encoding Mapping for FHLBankID:")
print(label_mapping)

df = df[~((df['Income'] == 1) & (df['Year'].isin([2010, 2011])))]
print("Removed rows where Income is 1 and Year is either 2010 or 2011 from")

df['CoAge'] = df['CoAge'].replace(98, 998)
print("replaced all of the value 98 with 998 in the CoAge")

df.loc[df['Year'] > 2018, 'Income'] *= 12
print("Converted Income to annualy ( newer years have it as monthly )")

df = df[df['Income'] > 10000]
print("Removed rows with Income <10000")

df = df[df['BoGender'] != 6]
df = df[df['CoGender'] != 6]
print("Removed rows with BoGender and CoGender == 6")

df = df[~((df['Co-Borrower Credit Score'] == 9) & (df['CoGender'] != 4))]
print("Removed rows where Co-Borrower Credit Score == 9 but CoGender is not 4")

df = df[~((df['CoGender'] == 4) & (df['CoAge'] != 998))]
print("Removed rows where CoGender == 4 but CoAge is not 998")

df = df[~((df['CoRace'] == 8) & (df['CoAge'] != 998))]
print("Removed rows where CoRace == 8 but CoAge is not 998")

df.loc[df['CoAge'] == 998, 'CoAge'] = 42
print("Replaced CoAge == 998 with 42(median)")

df.loc[df['CoGender'] == 4, 'Co-Borrower Credit Score'] = 9
print("Set Co-Borrower Credit Score to 9 where CoGender == 4")

df = df[df['Front'] < 900]
print("Removed rows where Front >= 900")

df = df.drop(columns=['Tract'])
print('Dropped column Tract')

sample_size = df['Year'].value_counts().min()
df = df.groupby('Year').apply(lambda x: x.sample(n=sample_size, random_state=42, replace=True)).reset_index(drop=True)
df['Year'].value_counts().sort_index()
print("Sampled based on Year and the minimum number of rows in a Year")

df.to_csv("Data/Sampled_data.csv", index=False)
print("Saved Sampled_data.csv in Data/")

Removed rows with missing values of subset=[BoAge, CoAge, Borrower Credit Score]
Imputed FHLBankID with RandomForestClassifier with Validation Accuracy : 0.9193
Label Encoding Mapping for FHLBankID:
{'Atlanta': np.int64(0), 'Boston': np.int64(1), 'Chicago': np.int64(2), 'Cincinnati': np.int64(3), 'Dallas': np.int64(4), 'Des Moines': np.int64(5), 'Indianapolis': np.int64(6), 'New York': np.int64(7), 'Pittsburgh': np.int64(8), 'San Francisco': np.int64(9), 'Topeka': np.int64(10)}
Removed rows where Income is 1 and Year is either 2010 or 2011 from
replaced all of the value 98 with 998 in the CoAge
Converted Income to annualy ( newer years have it as monthly )
Removed rows with Income <10000
Removed rows with BoGender and CoGender == 6
Removed rows where Co-Borrower Credit Score == 9 but CoGender is not 4
Removed rows where CoGender == 4 but CoAge is not 998
Removed rows where CoRace == 8 but CoAge is not 998
Replaced CoAge == 998 with 42(median)
Set Co-Borrower Credit Score to 9 where CoG

C:\Users\matinmarjani\AppData\Local\Temp\ipykernel_16068\2144758848.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Year').apply(lambda x: x.sample(n=sample_size, random_state=42, replace=True)).reset_index(drop=True)


Sampled based on Year and the minimum number of rows in a Year
Saved Sampled_data.csv in Data/


### Normalization

In [30]:
import pandas as pd

def z_score_normalization(df, columns):
    """
    Apply Z-score normalization to selected columns in a DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to normalize.
    
    Returns:
        pd.DataFrame: DataFrame with normalized selected columns.
    """
    df_normalized = df.copy()
    for col in columns:
        mean = df[col].mean()
        std = df[col].std()
        df_normalized[col] = (df[col] - mean) / std
    return df_normalized

df = pd.read_csv("merged_data_step_4_Encoded.csv")
columns_to_normalize = ['MinPer', 'TraMedY', 'LocMedY', 'Income', 'CurAreY', 'UPB', 'LTV', 'Term', 'BoAge', 'CoAge', 'Rate', 'Amount', 'Front', 'Back']

df_normalized = z_score_normalization(df, columns_to_normalize)
df_normalized.to_csv("merged_data_step_4_Encoded_normalized.csv", index=False)
df_normalized


,Year,MinPer,TraMedY,LocMedY,Income,CurAreY,UPB,LTV,MortDate,AcquDate,...,MSA_49420,MSA_49460,MSA_49500,MSA_49620,MSA_49660,MSA_49700,MSA_49740,MSA_49780,MSA_49820,MSA_99999
0,2009,-0.709019,-0.813342,-1.507925,-0.182313,-0.779598,0.461425,-0.532716,2009,2009,...,0,0,0,0,0,0,0,0,0,0
1,2009,-0.513915,-0.096767,-0.554048,0.053803,-0.223125,-0.158887,-0.532716,2009,2009,...,0,0,0,0,0,0,0,0,0,0
2,2009,-0.774054,-1.278908,-1.799946,-0.247034,-1.290951,-1.017128,-0.532716,2009,2009,...,0,0,0,0,1,0,0,0,0,0
3,2009,-0.774054,-0.634784,-0.554048,0.265153,-0.223125,-0.192877,-0.532716,2009,2009,...,0,0,0,0,0,0,0,0,0,0
4,2009,1.437124,-1.154904,-1.804670,-0.567118,-1.178152,-1.105501,-0.532716,2009,2009,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385510,2021,-0.578950,-0.381259,0.299811,-0.477536,0.122790,-1.123346,1.204068,2021,2021,...,0,0,0,0,0,0,0,0,0,1
385511,2021,-0.513915,1.326693,1.798586,0.370600,2.062924,-0.456297,0.477777,2021,2021,...,0,0,0,0,0,0,0,0,0,0
385512,2021,-0.058673,0.545288,1.081593,-0.110124,1.010138,0.043394,1.804048,2021,2021,...,0,0,0,0,0,0,0,0,0,0
385513,2021,-0.643985,-0.105008,-0.047550,1.206082,-0.305844,0.979769,1.867204,2020,2021,...,0,0,0,0,0,0,0,0,0,1


### ONE-HOT ENCODING

In [ ]:
import pandas as pd

# Assuming df is already loaded with the required data
# Calculating correlation between 'Borrower Credit Score' and 'Co-Borrower Credit Score'
df_sampled = pd.read_csv("merged_data_step_3_sampled.csv")
correlation = df['Borrower Credit Score'].corr(df['Co-Borrower Credit Score'])

# Display the correlation result
correlation


-0.030877536091840352

In [26]:
import pandas as pd

# Load dataset
df_droppedNA = pd.read_csv("merged_data_step_3_sampled.csv")

# Define categorical features
categorical_features = [
    'FHLBankID', 'Purpose', 'NumBor', 'BoRace', 'CoRace', 
    'BoGender', 'CoGender', 'Self', 'FIPSStateCode', 
    'FIPSCountyCode', 'MSA'
]
# categorical_features += ['Co-Borrower Credit Score', 'Borrower Credit Score']

# Apply one-hot encoding and convert boolean values to integers
df_encoded = pd.get_dummies(df_droppedNA, columns=categorical_features, drop_first=True).astype(int)

# Display results
print(df_encoded.head())
print(f"Original : {df_droppedNA.shape}, Encoded : {df_encoded.shape}")

# Save encoded dataset
df_encoded.to_csv("merged_data_step_4_Encoded.csv", index=False)


   Year  MinPer  TraMedY  LocMedY  Income  CurAreY     UPB  LTV  MortDate  \
0  2009       2    51140    40637   88245    58800  250000    0      2009   
1  2009       5    71920    54771  113100    66200  177000    0      2009   
2  2009       1    37639    36310   81432    52000   76000    0      2009   
3  2009       1    56318    54771  135348    66200  173000    0      2009   
4  2009      35    41235    36240   47738    53500   65600    0      2009   

   AcquDate  ...  MSA_49420  MSA_49460  MSA_49500  MSA_49620  MSA_49660  \
0      2009  ...          0          0          0          0          0   
1      2009  ...          0          0          0          0          0   
2      2009  ...          0          0          0          0          1   
3      2009  ...          0          0          0          0          0   
4      2009  ...          0          0          0          0          0   

   MSA_49700  MSA_49740  MSA_49780  MSA_49820  MSA_99999  
0          0          0    

In [32]:
import pandas as pd

# Load dataset
df_droppedNA = pd.read_csv("merged_data_step_3_sampled.csv")

# Define categorical features
categorical_features = [
    'FHLBankID', 'Purpose', 'NumBor', 'BoRace', 'CoRace', 
    'BoGender', 'CoGender', 'Self', 'FIPSStateCode', 
]

df_droppedNA.drop(columns=["MSA"])
df_droppedNA.drop(columns=["FIPSCountyCode"])

# categorical_features += ['Co-Borrower Credit Score', 'Borrower Credit Score']

# Apply one-hot encoding and convert boolean values to integers
df_encoded = pd.get_dummies(df_droppedNA, columns=categorical_features, drop_first=True).astype(int)

# Display results
print(df_encoded.head())
print(f"Original : {df_droppedNA.shape}, Encoded : {df_encoded.shape}")

# Save encoded dataset
df_encoded.to_csv("merged_data_step_4_Encoded_v2.csv", index=False)


import pandas as pd

def z_score_normalization(df, columns):
    """
    Apply Z-score normalization to selected columns in a DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to normalize.
    
    Returns:
        pd.DataFrame: DataFrame with normalized selected columns.
    """
    df_normalized = df.copy()
    for col in columns:
        mean = df[col].mean()
        std = df[col].std()
        df_normalized[col] = (df[col] - mean) / std
    return df_normalized

df = pd.read_csv("merged_data_step_4_Encoded_v2.csv")
columns_to_normalize = ['MinPer', 'TraMedY', 'LocMedY', 'Income', 'CurAreY', 'UPB', 'LTV', 'Term', 'BoAge', 'CoAge', 'Rate', 'Amount', 'Front', 'Back']

df_normalized = z_score_normalization(df, columns_to_normalize)
df_normalized.to_csv("merged_data_step_4_Encoded_normalized_v2.csv", index=False)
df_normalized



   Year  FIPSCountyCode    MSA  MinPer  TraMedY  LocMedY  Income  CurAreY  \
0  2009             163  19340       2    51140    40637   88245    58800   
1  2009             165  17140       5    71920    54771  113100    66200   
2  2009              85  49660       1    37639    36310   81432    52000   
3  2009              37  17140       1    56318    54771  135348    66200   
4  2009             139  34700      35    41235    36240   47738    53500   

      UPB  LTV  ...  FIPSStateCode_49  FIPSStateCode_50  FIPSStateCode_51  \
0  250000    0  ...                 0                 0                 0   
1  177000    0  ...                 0                 0                 0   
2   76000    0  ...                 0                 0                 0   
3  173000    0  ...                 0                 0                 0   
4   65600    0  ...                 0                 0                 0   

   FIPSStateCode_53  FIPSStateCode_54  FIPSStateCode_55  FIPSStateCode_56 

,Year,FIPSCountyCode,MSA,MinPer,TraMedY,LocMedY,Income,CurAreY,UPB,LTV,...,FIPSStateCode_49,FIPSStateCode_50,FIPSStateCode_51,FIPSStateCode_53,FIPSStateCode_54,FIPSStateCode_55,FIPSStateCode_56,FIPSStateCode_66,FIPSStateCode_72,FIPSStateCode_78
0,2009,163,19340,-0.709019,-0.813342,-1.507925,-0.182313,-0.779598,0.461425,-0.532716,...,0,0,0,0,0,0,0,0,0,0
1,2009,165,17140,-0.513915,-0.096767,-0.554048,0.053803,-0.223125,-0.158887,-0.532716,...,0,0,0,0,0,0,0,0,0,0
2,2009,85,49660,-0.774054,-1.278908,-1.799946,-0.247034,-1.290951,-1.017128,-0.532716,...,0,0,0,0,0,0,0,0,0,0
3,2009,37,17140,-0.774054,-0.634784,-0.554048,0.265153,-0.223125,-0.192877,-0.532716,...,0,0,0,0,0,0,0,0,0,0
4,2009,139,34700,1.437124,-1.154904,-1.804670,-0.567118,-1.178152,-1.105501,-0.532716,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385510,2021,129,99999,-0.578950,-0.381259,0.299811,-0.477536,0.122790,-1.123346,1.204068,...,0,0,0,0,0,1,0,0,0,0
385511,2021,17,37980,-0.513915,1.326693,1.798586,0.370600,2.062924,-0.456297,0.477777,...,0,0,0,0,0,0,0,0,0,0
385512,2021,163,41180,-0.058673,0.545288,1.081593,-0.110124,1.010138,0.043394,1.804048,...,0,0,0,0,0,0,0,0,0,0
385513,2021,65,99999,-0.643985,-0.105008,-0.047550,1.206082,-0.305844,0.979769,1.867204,...,0,0,0,0,0,0,0,0,0,0
